# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('../WeatherPy/Output_CSV_PNGs/city_weather.csv')
vacation_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,muhos,FI,64.81,25.99,35.294,74,11,8.70
1,rikitea,PF,-23.12,-134.97,76.658,79,82,3.61
2,saint-philippe,RE,-21.36,55.77,80.924,54,3,2.10
3,taolanaro,PK,26.00,63.04,92.768,17,0,3.52
4,turbat,NZ,-46.19,168.86,53.006,88,100,6.71
...,...,...,...,...,...,...,...,...
538,auch,CN,47.87,88.12,59.108,31,12,0.63
539,qui nhon,ID,-5.32,119.37,87.800,66,20,4.10
540,catio,JP,26.18,127.76,65.372,72,75,5.70
541,jinchang,PT,32.67,-17.10,63.194,82,75,4.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Locations
locations = vacation_df[["Latitude", "Longitude"]]
humidity = vacation_df["Humidity (%)"].astype(float)


In [4]:

#create a blank canvas of google maps
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=100,point_radius = 2)

# add layer to the figure
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# arrive at narrowed down cities fitting the conditions
condition1 = vacation_df[vacation_df["Temperature (F)"] > 70]
condition2 = condition1[condition1["Temperature (F)"] < 80]
condition3 = condition2[condition2['Wind Speed (mph)'] < 10]
narrowed_city_df = condition3[condition3['Cloudiness (%)'] == 0] 
vacay_places = narrowed_city_df.drop( columns = 'Humidity (%)')

# reset Index
indexed_vacay_places = vacay_places.reset_index()
del indexed_vacay_places["index"]
indexed_vacay_places

,City,Country,Latitude,Longitude,Temperature (F),Cloudiness (%),Wind Speed (mph)
0,chuy,CN,30.63,117.85,71.996,0,0.42
1,ossora,ML,22.68,-3.98,75.200,0,5.88
2,sinnamary,CN,42.80,93.45,76.388,0,2.91
3,bethel,ML,14.46,-4.92,75.344,0,3.53
4,tenenkou,GM,13.35,-15.75,73.004,0,2.81
5,medicine hat,AU,-34.60,138.73,70.286,0,3.13
6,moiyabana,AO,-15.20,12.15,75.182,0,4.15
7,mahon,MR,20.50,-10.07,73.706,0,4.13
8,damietta,MR,20.52,-13.05,75.650,0,4.26
9,darhan,AU,-32.50,137.77,74.102,0,5.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Hotel variable
hotel_df = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_vacay_places["City"])):

    lat = indexed_vacay_places.loc[city]["Latitude"]
    lng = indexed_vacay_places.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params = params)
    hotel_response = hotel_request.json()

    try:
        hotel_df.append(hotel_response["results"][0]["name"])
    except:
        hotel_df.append("No Hotels in the radius")

#Dataframe with nearest hotel
indexed_vacay_places["Hotel Name"] = hotel_df
indexed_vacay_places
   

,City,Country,Latitude,Longitude,Temperature (F),Cloudiness (%),Wind Speed (mph),Hotel Name
0,chuy,CN,30.63,117.85,71.996,0,0.42,Shell Shandong Liaocheng Development District ...
1,ossora,ML,22.68,-3.98,75.200,0,5.88,No Hotels in the radius
2,sinnamary,CN,42.80,93.45,76.388,0,2.91,Laoyutang Resort
3,bethel,ML,14.46,-4.92,75.344,0,3.53,No Hotels in the radius
4,tenenkou,GM,13.35,-15.75,73.004,0,2.81,No Hotels in the radius
5,medicine hat,AU,-34.60,138.73,70.286,0,3.13,Eagle Foundry Bed & Breakfast
6,moiyabana,AO,-15.20,12.15,75.182,0,4.15,Chik-Chik Namibe
7,mahon,MR,20.50,-10.07,73.706,0,4.13,No Hotels in the radius
8,damietta,MR,20.52,-13.05,75.650,0,4.26,Oumou Elghoura
9,darhan,AU,-32.50,137.77,74.102,0,5.10,Majestic Oasis Apartments


In [10]:
 # NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_vacay_places.iterrows()]
locations = indexed_vacay_places[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Hotel Name: {hotel}" for hotel in hotel_df])
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))